### The protocol files used are generated using utils_poc.ipynb file in SASVC2022_Baseline repo

In [1]:
config = {
#     "database_path": "/DATA/nfsshare/rishith/datasets/asvSpoof2019/DS_10283_3336/LA/",
#     "asv_score_path": "ASVspoof2019_LA_asv_scores/ASVspoof2019.LA.asv.eval.gi.trl.scores.txt",
    "model_path": "./exp_result/LA_AASIST-L_ep20_bs24/weights/best.pth",
    "batch_size": 24,
    "num_epochs": 20,
    "loss": "CCE",
    "track": "LA",
    "eval_all_best": "True",
    "eval_output": "eval_scores_using_best_dev_model_asv_libri.txt",
    "cudnn_deterministic_toggle": "True",
    "cudnn_benchmark_toggle": "False",
    "model_config": {
        "architecture": "AASIST",
        "nb_samp": 64600,
        "first_conv": 128,
        "filts": [70, [1, 32], [32, 32], [32, 24], [24, 24]],
        "gat_dims": [24, 32],
        "pool_ratios": [0.4, 0.5, 0.7, 0.5],
        "temperatures": [2.0, 2.0, 100.0, 100.0]
    },
    "optim_config": {
        "optimizer": "adam", 
        "amsgrad": "False",
        "base_lr": 0.0001,
        "lr_min": 0.000005,
        "betas": [0.9, 0.999],
        "weight_decay": 0.0001,
        "scheduler": "cosine"
    }
}

In [2]:
# load experiment configurations
# with open(args.config, "r") as f_json:
#     config = json.loads(f_json.read())
model_config = config["model_config"]
optim_config = config["optim_config"]
optim_config["epochs"] = config["num_epochs"]
track = config["track"]
assert track in ["LA", "PA", "DF"], "Invalid track given"
if "eval_all_best" not in config:
    config["eval_all_best"] = "True"
if "freq_aug" not in config:
    config["freq_aug"] = "False"

In [3]:
import json
import os
import sys
import warnings
from importlib import import_module
from pathlib import Path
from shutil import copy
from typing import Dict, List, Union

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchcontrib.optim import SWA

from data_utils import (Dataset_ASVspoof2019_train,
                        Dataset_ASVspoof2019_devNeval, genSpoof_list, Dataset_PrevDbs)
from evaluation import calculate_tDCF_EER
from utils import create_optimizer, seed_worker, set_seed, str_to_bool

In [4]:
# make experiment reproducible
seed = 1234
set_seed(seed, config)

In [5]:
# define database related paths
output_dir = './exp_result'
output_dir = Path(output_dir)
# prefix_2019 = "ASVspoof2019.{}".format(track)
# database_path = Path(config["database_path"])

eval_trial_path = '/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/libri_test_protocol.txt'
# (
#         database_path /
#         "ASVspoof2019_{}_cm_protocols/{}.cm.eval.trl.txt".format(
#             track, prefix_2019))

In [6]:
# define model related paths
model_tag = "{}_{}_ep{}_bs{}".format(
        track,
        'AASIST-L',
        config["num_epochs"], config["batch_size"])
# if args.comment:
#     model_tag = model_tag + "_{}".format(acomment)
model_tag = output_dir / model_tag
model_save_path = model_tag / "weights"
eval_score_path = model_tag / config["eval_output"]
writer = SummaryWriter(model_tag)
os.makedirs(model_save_path, exist_ok=True)
# copy(config, model_tag / "config.conf")

In [7]:
# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))

Device: cuda


In [8]:
from main import get_model

In [9]:
# define model architecture
model = get_model(model_config, device)

no. model params:85306


In [10]:
# list of dataset partitions
SET_PARTITION = ["trn", "eval"]

# list of countermeasure(CM) protocols
SET_CM_PROTOCOL = {
    "trn": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/libri_train_protocol.txt",
    "eval": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/libri_test_protocol.txt",
}

# directories of each dataset partition
SET_DIR = {
    "trn": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/Libri/train/",
    "eval": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/Libri/test/",
}

In [11]:
# Dataloader
meta_lines = open(SET_CM_PROTOCOL["eval"], "r").readlines()
utt_list = []
for line in meta_lines:
    tmp = line.strip().split(" ")

    utt = tmp[0]
    utt_list.append(utt)

base_dir = SET_DIR['eval']
dataset = Dataset_PrevDbs(utt_list, Path(base_dir))
loader = DataLoader(
        dataset, batch_size=30, shuffle=False, drop_last=False, pin_memory=True
    )

In [12]:
# evaluates pretrained model and exit script
    
model.load_state_dict(
            torch.load(config["model_path"], map_location=device))
print("Model loaded : {}".format(config["model_path"]))
model.eval()

Model loaded : ./exp_result/LA_AASIST-L_ep20_bs24/weights/best.pth


Model(
  (conv_time): CONV()
  (first_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=True)
  (drop_way): Dropout(p=0.2, inplace=True)
  (selu): SELU(inplace=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): Residual_block(
        (conv1): Conv2d(1, 32, kernel_size=(2, 3), stride=(1, 1), padding=(1, 1))
        (selu): SELU(inplace=True)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(2, 3), stride=(1, 1), padding=(0, 1))
        (conv_downsample): Conv2d(1, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (mp): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Sequential(
      (0): Residual_block(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(2, 3),

In [13]:
eval_score_path

PosixPath('exp_result/LA_AASIST-L_ep20_bs24/eval_scores_using_best_dev_model_asv_libri.txt')

In [14]:
trial_path = SET_CM_PROTOCOL['eval']
with open(trial_path, "r") as f_trl:
    trial_lines = f_trl.readlines()
fname_list = []
score_list = []
for batch_x, utt_id in loader:
    batch_x = batch_x.to(device)
    with torch.no_grad():
        _, batch_out = model(batch_x)
        batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
    # add outputs
    fname_list.extend(utt_id)
    score_list.extend(batch_score.tolist())   
assert len(trial_lines) == len(fname_list) == len(score_list)
with open(eval_score_path, "w") as fh:
    for fn, sco, trl in zip(fname_list, score_list, trial_lines):
        utt_id, key = trl.strip().split(' ')
        assert fn == utt_id
        fh.write("{} {} {}\n".format(utt_id, key, sco))

# EER

In [16]:
import numpy as np
from evaluation import compute_eer

In [17]:
cm_data = np.genfromtxt(eval_score_path, dtype=str)
cm_data[:5, :]

array([['251_137823_000065_000000', 'bonafide', '-0.3879561722278595'],
       ['84_121550_000256_000000', 'bonafide', '-0.19817444682121277'],
       ['84_121550_000276_000000', 'bonafide', '1.0496079921722412'],
       ['84_121550_000281_000000', 'bonafide', '1.4189954996109009'],
       ['251_137823_000060_000002', 'bonafide', '0.7846293449401855']],
      dtype='<U28')

In [18]:
cm_keys = cm_data[:, 1]
cm_scores = cm_data[:, 2].astype(float)

In [19]:
bona_cm = cm_scores[cm_keys == 'bonafide']
print(bona_cm.shape)

spoof_cm = cm_scores[cm_keys == 'spoof']
print(spoof_cm.shape)

(42,)
(42,)


In [20]:
eer_cm = compute_eer(bona_cm, spoof_cm)[0]
print(eer_cm * 100)

40.476190476190474


For swa.pth

In [21]:
model.load_state_dict(
            torch.load('./exp_result/LA_AASIST-L_ep20_bs24/weights/swa.pth', map_location=device))
print("Model loaded : {}".format('./exp_result/LA_AASIST-L_ep20_bs24/weights/swa.pth'))
model.eval()

Model loaded : ./exp_result/LA_AASIST-L_ep20_bs24/weights/swa.pth


Model(
  (conv_time): CONV()
  (first_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=True)
  (drop_way): Dropout(p=0.2, inplace=True)
  (selu): SELU(inplace=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): Residual_block(
        (conv1): Conv2d(1, 32, kernel_size=(2, 3), stride=(1, 1), padding=(1, 1))
        (selu): SELU(inplace=True)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(2, 3), stride=(1, 1), padding=(0, 1))
        (conv_downsample): Conv2d(1, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (mp): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Sequential(
      (0): Residual_block(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(2, 3),

In [22]:
trial_path = SET_CM_PROTOCOL['eval']
with open(trial_path, "r") as f_trl:
    trial_lines = f_trl.readlines()
fname_list = []
score_list = []
for batch_x, utt_id in loader:
    batch_x = batch_x.to(device)
    with torch.no_grad():
        _, batch_out = model(batch_x)
        batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
    # add outputs
    fname_list.extend(utt_id)
    score_list.extend(batch_score.tolist())   
assert len(trial_lines) == len(fname_list) == len(score_list)
with open('exp_result/LA_AASIST-L_ep20_bs24/eval_scores_using_best_dev_model_asv_libri_swa.txt', "w") as fh:
    for fn, sco, trl in zip(fname_list, score_list, trial_lines):
        utt_id, key = trl.strip().split(' ')
        assert fn == utt_id
        fh.write("{} {} {}\n".format(utt_id, key, sco))

In [23]:
cm_data = np.genfromtxt('exp_result/LA_AASIST-L_ep20_bs24/eval_scores_using_best_dev_model_asv_libri_swa.txt', dtype=str)
cm_keys = cm_data[:, 1]
cm_scores = cm_data[:, 2].astype(float)
bona_cm = cm_scores[cm_keys == 'bonafide']
print(bona_cm.shape)
spoof_cm = cm_scores[cm_keys == 'spoof']
print(spoof_cm.shape)

eer_cm = compute_eer(bona_cm, spoof_cm)[0]
print(eer_cm * 100)

(42,)
(42,)
40.476190476190474


pre-trained AASIST-L

In [27]:
model.load_state_dict(
            torch.load('./models/weights/AASIST-L.pth', map_location=device))
print("Model loaded : {}".format('./models/weights/AASIST-L.pth'))
model.eval()

Model loaded : ./models/weights/AASIST-L.pth


Model(
  (conv_time): CONV()
  (first_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=True)
  (drop_way): Dropout(p=0.2, inplace=True)
  (selu): SELU(inplace=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): Residual_block(
        (conv1): Conv2d(1, 32, kernel_size=(2, 3), stride=(1, 1), padding=(1, 1))
        (selu): SELU(inplace=True)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(2, 3), stride=(1, 1), padding=(0, 1))
        (conv_downsample): Conv2d(1, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (mp): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Sequential(
      (0): Residual_block(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(2, 3),

In [28]:
trial_path = SET_CM_PROTOCOL['eval']
with open(trial_path, "r") as f_trl:
    trial_lines = f_trl.readlines()
fname_list = []
score_list = []
for batch_x, utt_id in loader:
    batch_x = batch_x.to(device)
    with torch.no_grad():
        _, batch_out = model(batch_x)
        batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
    # add outputs
    fname_list.extend(utt_id)
    score_list.extend(batch_score.tolist())   
assert len(trial_lines) == len(fname_list) == len(score_list)
with open('exp_result/LA_AASIST-L_ep20_bs24/asv_libri_pretrained_aasist-L.txt', "w") as fh:
    for fn, sco, trl in zip(fname_list, score_list, trial_lines):
        utt_id, key = trl.strip().split(' ')
        assert fn == utt_id
        fh.write("{} {} {}\n".format(utt_id, key, sco))

In [29]:
cm_data = np.genfromtxt('exp_result/LA_AASIST-L_ep20_bs24/asv_libri_pretrained_aasist-L.txt', dtype=str)
cm_keys = cm_data[:, 1]
cm_scores = cm_data[:, 2].astype(float)
bona_cm = cm_scores[cm_keys == 'bonafide']
print(bona_cm.shape)
spoof_cm = cm_scores[cm_keys == 'spoof']
print(spoof_cm.shape)

eer_cm = compute_eer(bona_cm, spoof_cm)[0]
print(eer_cm * 100)

(42,)
(42,)
35.714285714285715
